In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import glob
import sklearn
import pandas as pd
import os
import sys
import time
import re
import seaborn as sns

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, cohen_kappa_score, precision_score, recall_score, accuracy_score
from joblib import dump, load
from sklearn.feature_selection import SelectFromModel, SelectPercentile, VarianceThreshold, chi2, f_classif

plt.rc('figure', figsize=(16, 9))

from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import roc_curve, auc
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, cohen_kappa_score, precision_score, recall_score, accuracy_score

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# read in raw files
filenames=glob.glob("SNP_data_raw/5_95_data/final_data/*.csv")

In [ ]:
filenames

In [ ]:
gffs_list=[]
gffs_list.append('../Bioinfor2/gff/azt_ref.gff')
gffs_list.append('../Bioinfor2/gff/cip_ref.gff')
gffs_list.append('../Bioinfor2/gff/lev_ref.gff')
gffs_list.append('../Bioinfor2/gff/cefo_ref.gff')
gffs_list.append('../Bioinfor2/gff/mer_ref.gff')
gffs_list.append('../Bioinfor2/gff/amp_ref.gff')
gffs_list.append('../Bioinfor2/gff/tob_ref.gff')
gffs_list.append('../Bioinfor2/gff/pip_ref.gff')
gffs_list.append('../Bioinfor2/gff/cefe_ref.gff')
gffs_list.append('../Bioinfor2/gff/tet_ref.gff')
gffs_list.append('../Bioinfor2/gff/cefu_ref.gff')
gffs_list.append('../Bioinfor2/gff/gen_ref.gff')
gffs_list.append('../Bioinfor2/gff/tri_ref.gff')

In [ ]:
def gff_to_df(gff_file):
    with open(gff_file) as f:
        content = f.readlines()

    content = [x.strip() for x in content]
    content_new=[]

    for i in content:
        if i=='##FASTA':
            break
        content_new.append(i)

    content_list=[x.rstrip().split('\t') for x in content_new[2:]]    

    df=pd.DataFrame(content_list) 
    df.columns = ['a', 'b','c','d','e','f','g','h','i']

    df=df[df.c=='gene']
    a_list=[]
    for x in df.i:
        x=x.split(';')
        if(len(x)==4):
            a_list.append(x[2].split('=')[1])
        else:
            a_list.append(np.array([0])

    df['gene']=a_list   
    df.d=df.d.astype('int64')
    df.e=df.e.astype('int64')

    df=df[['d','e','gene']].rename(columns={"d": "start", "e": "end"})
    return df

In [ ]:
def top_weights_colormap(sim_all_df_T, top_num, X, y):

    sim_all_df_Prod = sim_all_df_T.sort_values("svm_weight_sum", ascending=False)[:top_num]
    sim_all_df_Prod.drop("svm_weight_sum", axis=1, inplace=True)

    allele_to_colormap = []

    clust_alleles = list(sim_all_df_T.sort_values("svm_weight_sum", ascending=False)[:top_num].index)

    allele_df = X.copy()[clust_alleles]
    suscept_y = y[y==0].copy()
    resist_y = y[y==1].copy()
    suscept_alleles = allele_df.loc[suscept_y.index, :].copy()
    resist_alleles = allele_df.loc[resist_y.index, :].copy()


    allele_to_colormap = []
    for allele in resist_alleles.columns:
        allele_r_percent = round(resist_alleles[allele].sum()/float(allele_df[allele].sum()),2)
        allele_to_colormap.append(sns.diverging_palette(250, 10, n=100)[int(allele_r_percent*100-1)])

    return sim_all_df_Prod, allele_to_colormap, clust_alleles

In [ ]:
def plot_heatmap(tmp_db, fgsz, allele_to_colormap, cmap_choose):
    g = sns.clustermap(tmp_db, method = "complete", metric="cityblock",
                       yticklabels=True, xticklabels=True,
                       cmap = cmap_choose,
                       linewidths=.1,
                       row_colors = allele_to_colormap,
                       col_colors = allele_to_colormap,
                       figsize=fgsz)
    # figsize=(22,4)
    plt.setp(g.ax_heatmap.yaxis.get_majorticklabels(), rotation=0);
    plt.setp(g.ax_heatmap.xaxis.get_majorticklabels(), rotation=90);
    return g

In [ ]:
file_gffs=dict(zip(filenames, gffs_list))

In [ ]:
file_gffs

In [ ]:
folder='SVMs_paper3'
import os
if not os.path.exists(folder):
    os.makedirs(folder)

In [ ]:
import pickle
with open('anti_hyperparams_svm_ecoli.pickle', 'rb') as handle:
    anti_hyperparams_svm = pickle.load(handle)

In [ ]:
# top_num: number of highly associated alleles to consider for downstream analysis 1:R
top_num=50
top_allele_weighted=200
top_allele_pairs=100

for k,v in file_gffs.items():
    accuracy_list=[]
    precision_score_list=[]
    recall_score_list=[]
    roc_auc_score_list=[]
    cohenkappa_score_list=[]
    tn_list=[]
    fp_list=[]
    fn_list=[]
    tp_list=[]
    specificity_list=[]
    
    antibiotic_name=k.split('/')[-1].split('_')[0]
    
    data=pd.read_csv(k)
    X=data.iloc[:,1:-1]
    y=data.iloc[:,-1]
    sim_all_df = pd.DataFrame()

    print('processing {}...'.format(antibiotic_name))
    C_tunned=anti_hyperparams_svm[antibiotic_name]['clf__C']



    for r in range(1,51):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=r)
        
    
        model = Pipeline([
          ('scaler', StandardScaler()),
          ('clf', SVC(kernel='linear',C=C_tunned))
        ])
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        predictions = [round(value) for value in y_pred]
        
#         sim_t_df = pd.DataFrame([model['clf'].coef_[0]], columns=X.columns[model['fs'].get_support()])
        sim_t_df = pd.DataFrame([model['clf'].coef_[0]], columns=X.columns)

        sim_all_df = pd.concat([sim_all_df, sim_t_df], ignore_index=True)
        
        accuracy_list.append(accuracy_score(y_test, predictions))
        precision_score_list.append(precision_score(y_test,predictions))
        recall_score_list.append(recall_score(y_test,predictions))
        try:
            roc_auc_score_list.append(roc_auc_score(y_test,predictions))
        except ValueError:
            roc_auc_score_list.append(-1)
        cohenkappa_score_list.append(cohen_kappa_score(y_test,predictions))
        tn, fp, fn, tp=confusion_matrix(y_test, predictions, labels=[0,1]).ravel()
        tn_list.append(tn)
        fp_list.append(fp)
        fn_list.append(fn)
        tp_list.append(tp)
        specificity_list.append(tn / (tn+fp))


    sim_all_df_T = sim_all_df.transpose().copy()
    sim_all_df_T["svm_weight_sum"] = sim_all_df_T.apply(lambda x: abs(x).sum(), axis=1)
    sim_all_df_T_top = sim_all_df_T.sort_values("svm_weight_sum", ascending=False)[:top_num]
    sim_all_df_T_top.to_csv("{}/{}_SVMs_importances.csv".format(folder,antibiotic_name))

    sim_all_df_T_top.drop("svm_weight_sum", axis=1, inplace=True)
    sim_corr_df = sim_all_df_T_top.fillna(0).T.corr().copy()    
    sim_corr_df = sim_corr_df[abs(sim_corr_df)>.05].fillna(0)
    sim_corr_df.to_csv("{}/{}_correlation_matrix.csv".format(folder,antibiotic_name))
    
    top_weighted_alleles = list(sim_all_df_T_top.index[:top_allele_weighted])
    sim_corr_df = sim_corr_df.loc[top_weighted_alleles, top_weighted_alleles].copy()

    
    abs_corr_df = sim_corr_df.abs()
    os = (abs_corr_df.where(np.triu(np.ones(abs_corr_df.shape), k=1).astype(np.bool))
             .stack()
             .sort_values(ascending=False))
    
    sig_allele_interact_list = []
    for (allele_1, allele_2) in os.index[:top_allele_pairs]:
        if allele_1 in top_weighted_alleles or allele_2 in top_weighted_alleles:
                                    sig_allele_interact_list.append((allele_1, allele_2))
    pd.DataFrame(sig_allele_interact_list).to_csv(
        "{}/{}_sig_allele_interact_list.csv".format(folder,antibiotic_name))
    
    df=gff_to_df(v)
    sig_gene_interact_list=[]
    for (allele_1,allele_2) in sig_allele_interact_list:
        sig_gene_interact_list.append(
            (df[(df.start<int(allele_1)) & (df.end>int(allele_1))].gene.values,
            df[(df.start<int(allele_2)) & (df.end>int(allele_2))].gene.values)
        )
    pd.DataFrame(sig_gene_interact_list).to_csv("{}/{}_sig_gene_interact_list.csv".format(folder, antibiotic_name))
    

    stats = pd.DataFrame({'accuracy': accuracy_list, 
                              'precision': precision_score_list,
                              'specificity':specificity_list,
                              'sensitivity': recall_score_list,
                              'auc': roc_auc_score_list,
                              'kappa': cohenkappa_score_list,
                                 'tn':tn_list,
                                 'fp':fp_list,
                                 'fn':fn_list,
                                 'tp':tp_list
                         })

    stats.to_csv('{}/{}_SVM_stats.csv'.format(folder,antibiotic_name))
    
    sim_all_df_Prod, allele_to_colormap, clust_alleles=top_weights_colormap(sim_all_df_T, top_num, X, y)
    sim_corr_df = sim_all_df_Prod.fillna(0).T.corr().copy()
    sim_corr_df = sim_corr_df[abs(sim_corr_df)>.05].fillna(0)
    gene_of_snp=[]
    for snp in sim_corr_df.index:
        gene=df[(df.start<int(snp)) & (df.end>int(snp))].gene.values
        if gene!=0:
            gene_of_snp.append(gene[0])
        else:
            gene_of_snp.append(snp)
    sim_corr_df.index=gene_of_snp
    sim_corr_df.columns=gene_of_snp
    g_plot = plot_heatmap(sim_corr_df, (15, 15), allele_to_colormap, "seismic")
    g_plot.savefig('{}/{}_correlation.png'.format(folder,antibiotic_name))  

In [ ]:
sim_all_df_T_top

In [ ]:
g = sns.clustermap(sim_all_df_Prod.fillna(0), 
                   col_cluster=False, row_cluster=True, cmap="seismic",
                  figsize=(15,15), xticklabels=False)
plt.setp(g.ax_heatmap.yaxis.get_majorticklabels(), rotation=0);
plt.setp(g.ax_heatmap.yaxis.get_majorticklabels(), rotation=0);
# g.savefig(SUPP_SVM_PLANES+drug_name+"_"+estimator_type+"_"+SGD_or_GD+"_iterations.png")